In [ ]:
!pip install tableauhyperapi

In [1]:
import yfinance as yf
import pandas as pd
from tableauhyperapi import HyperProcess, Connection, TableDefinition, SqlType, Telemetry, CreateMode, Inserter

In [2]:
def dataframe_to_hyper(df, table_name, hyper_name):
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint, database=hyper_name, create_mode=CreateMode.CREATE_AND_REPLACE) as connection:
            
            columns = []
            for col_name in df.columns:
                if df[col_name].dtype == 'O':
                    col_type = SqlType.text()
                elif df[col_name].dtype == 'datetime64[ns]':
                    col_type = SqlType.timestamp()
                else:
                    col_type = SqlType.double()
                columns.append(TableDefinition.Column(name=col_name, type=col_type))
            table_definition = TableDefinition(table_name=table_name, columns=columns)
            connection.catalog.create_table(table_definition)
            

            with Inserter(connection, table_definition) as inserter:
                inserter.add_rows(rows=df.values)
                inserter.execute()

In [3]:
df = yf.download('^GSPC', start='2018-08-01', end='2023-09-16')

[*********************100%***********************]  1 of 1 completed


In [4]:
df = df.reset_index()

In [5]:
dataframe_to_hyper(df, "SampleTable", "Files/spx500.hyper")